In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the monthly data
dm = pd.read_csv('time_series_monthly.csv')
dm.rename(columns={
    'date': 'Date',
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. volume': 'Volume'
}, inplace=True)
dm['Date'] = pd.to_datetime(dm['Date'])
dm.sort_values('Date', ascending=True, inplace=True)

# Load and preprocess the daily data
dd = pd.read_csv('time_series.csv')
dd.rename(columns={
    'date': 'Date',
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. volume': 'Volume'
}, inplace=True)
dd['Date'] = pd.to_datetime(dd['Date'])
dd.sort_values('Date', ascending=True, inplace=True)

# Initialize lists to store buy and sell prices, and trade history
buy_price = []
trade_history = []

# Iterate through monthly data (dm) to check the previous month's low
for i in range(1, len(dm)):
    prev_month_low = float(dm.iloc[i-1]['Low'])  # Previous month's low value
    threshold = prev_month_low * 1.1
    
    # Extract the year and month for the current month
    curr_year, curr_month = pd.to_datetime(dm.iloc[i]['Date']).year, pd.to_datetime(dm.iloc[i]['Date']).month
    
    # Filter daily data for the next month in dd
    next_month_data = dd[(dd['Date'].dt.year == curr_year) & (dd['Date'].dt.month == curr_month)]
    
    # Check for buy and sell signals
    for _, row in next_month_data.iterrows():
        # Check for buy signal
        if row['Low'] <= threshold:
            buy_price.append([row['Date'], row['Low']])
            print(f"Buy signal on {row['Date']} with a low of {row['Low']}")
        
        # Check for corresponding sell signals
        if len(buy_price) > 0:
            for buy in buy_price:
                if row['High'] >= buy[1] * 1.1:
                    buy_date = buy[0]
                    buy_val = buy[1]
                    sell_date = row['Date']
                    sell_val = row['High']
                    
                    # Add to trade history and calculate holding period in months
                    months_diff = (sell_date.year - buy_date.year) * 12 + (sell_date.month - buy_date.month)
                    trade_history.append([buy_date, sell_date, buy_val, sell_val, months_diff])
                    print(f"Sell signal on {row['Date']} with a high of {row['High']}. Holding period: {months_diff} months.")
                    
                    # Remove buy from list after trade is completed
                    buy_price.remove(buy)
                    break

# Convert trade history into DataFrame
trade_history_df = pd.DataFrame(trade_history, columns=['Buy Date', 'Sell Date', 'Buy Price', 'Sell Price', 'Holding Period (Months)'])

# Insert empty rows where the month ends
for i in range(len(trade_history_df) - 1, 0, -1):
    curr_month = pd.to_datetime(trade_history_df.iloc[i]['Sell Date']).month
    prev_month = pd.to_datetime(trade_history_df.iloc[i - 1]['Sell Date']).month
    if curr_month != prev_month:
        # Insert an empty row after a month ends
        empty_row = pd.Series([np.nan] * trade_history_df.shape[1], index=trade_history_df.columns)
        trade_history_df = pd.concat([trade_history_df.iloc[:i+1], empty_row.to_frame().T, trade_history_df.iloc[i+1:]], ignore_index=True)

# Save trade history with empty rows to CSV
trade_history_df.to_csv('trade_history.csv', index=False)

# Plot Buy and Sell Prices
if not trade_history_df.empty:
    plt.figure(figsize=(10, 6))
    
    # Plot all daily close prices
    plt.plot(dd['Date'], dd['Close'], label="Daily Close Price", color="gray", alpha=0.5)
    
    # Highlight buy and sell points
    plt.scatter(pd.to_datetime(trade_history_df['Buy Date']), trade_history_df['Buy Price'], label='Buy', marker='^', color='green', s=100)
    plt.scatter(pd.to_datetime(trade_history_df['Sell Date']), trade_history_df['Sell Price'], label='Sell', marker='v', color='red', s=100)
    
    plt.title('Buy and Sell Signals with Holding Period')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()

print(trade_history_df)

trade_history_df.to_csv('trade_history.csv')

Empty DataFrame
Columns: [Buy Date, Sell Date, Buy Price, Sell Price, Holding Period (Months)]
Index: []


In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the daily data
daily_data = pd.read_csv('time_series.csv')
daily_data.rename(columns={
    'date': 'Date',
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. volume': 'Volume'
}, inplace=True)
daily_data['Date'] = pd.to_datetime(daily_data['Date'])
daily_data.sort_values('Date', ascending=True, inplace=True)

# Feature Engineering: Create lag features for previous day data
daily_data['Prev_Open'] = daily_data['Open'].shift(1)
daily_data['Prev_Close'] = daily_data['Close'].shift(1)
daily_data['Prev_High'] = daily_data['High'].shift(1)
daily_data['Prev_Low'] = daily_data['Low'].shift(1)
daily_data['Prev_Volume'] = daily_data['Volume'].shift(1)

# Drop rows with NaN values (as they don't have previous day data)
daily_data.dropna(inplace=True)

# Load the historical trade data (for training purposes)
trade_history_df = pd.read_csv('trade_history.csv')

# Feature Engineering: Extract useful features from the buy/sell data
trade_history_df['Buy Date'] = pd.to_datetime(trade_history_df['Buy Date'])
trade_history_df['Sell Date'] = pd.to_datetime(trade_history_df['Sell Date'])

# Merge daily data with trade history to incorporate lag and current day's features
trade_history_df = pd.merge(trade_history_df, daily_data, left_on='Buy Date', right_on='Date', how='inner')

# Create additional features based on the lag and current day data
trade_history_df['Price Change'] = trade_history_df['Sell Price'] - trade_history_df['Buy Price']
trade_history_df['Holding Period (Months)'] = (trade_history_df['Sell Date'] - trade_history_df['Buy Date']).dt.days / 30
trade_history_df['Price Change %'] = (trade_history_df['Price Change'] / trade_history_df['Buy Price']) * 100

# Target classification: Based on historical trade data, determine Buy (1), Sell (2), or Hold (0)
def classify_action(row):
    if row['Price Change %'] > 10:  # Condition for a profitable trade to Sell
        return 2  # Sell
    elif row['Price Change %'] < -5:  # Condition for a loss to Buy again (lower price)
        return 1  # Buy
    else:
        return 0  # Hold

trade_history_df['Action'] = trade_history_df.apply(classify_action, axis=1)

# Prepare the feature set and target
X = trade_history_df[['Buy Price', 'Open', 'Close', 'Prev_Open', 'Prev_Close', 'Prev_High', 'Prev_Low', 'Prev_Volume']]
y = trade_history_df['Action']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier for multi-class classification
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)



# Calculate the closest buy price
def calculate_closest_buy_price(row):
    low = row['Low']
    volume = row['Volume']
    open_price = row['Open']
    close_price = row['Close']
    
    # If today's low price is greater than the threshold, suggest a new buy price
    if open_price <= low:
        suggested_buy_price = low
    else:
        # Calculate a new price based on volume (example calculation, adjust as needed)
        suggested_buy_price = low + (volume / 100000)  # Example: Adding a small increment based on volume
    
    return suggested_buy_price



ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [35]:
# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

# Now, let's use the trained model to predict a new action (Buy, Hold, Sell) based on a new day's data
new_trade = pd.DataFrame({
    'Buy Price': [daily_data['Close'][0]],
    'Open': [daily_data['Open'][0]],
    'Close': [daily_data['Close'][0]],
    'Prev_Open': [daily_data['Prev_Open'][0]],
    'Prev_Close': [daily_data['Prev_Close'][0]],
    'Prev_High': [daily_data['Prev_High'][0]],
    'Prev_Low': [daily_data['Prev_Low'][0]],
    'Prev_Volume': [daily_data['Prev_Volume'][0]]
})
# Predict action (0: Hold, 1: Buy, 2: Sell)
predicted_action = clf.predict(new_trade)
action_dict = {0: 'Hold', 1: 'Buy', 2: 'Sell'}
print("Predicted Action:", action_dict[predicted_action[0]])


Model Accuracy: 100.00%
Classification Report:
               precision    recall  f1-score   support

           2       1.00      1.00      1.00         7

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7

Predicted Action: Sell


In [36]:
# Apply the function to daily data to get suggested buy prices
daily_data['Suggested_Buy_Price'] = daily_data.apply(calculate_closest_buy_price, axis=1)

# Display suggested buy prices for today
today_data = daily_data[daily_data['Date'] == pd.to_datetime('today')]
print("Suggested Buy Prices for Today:\n", today_data[['Date', 'Open', 'Close', 'Low', 'High', 'Volume', 'Suggested_Buy_Price']])


Suggested Buy Prices for Today:
 Empty DataFrame
Columns: [Date, Open, Close, Low, High, Volume, Suggested_Buy_Price]
Index: []


In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Load the daily data
daily_data = pd.read_csv('time_series.csv')
daily_data.rename(columns={
    'date': 'Date',
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. volume': 'Volume'
}, inplace=True)
daily_data['Date'] = pd.to_datetime(daily_data['Date'])
daily_data.sort_values('Date', ascending=True, inplace=True)

# Feature Engineering: Create lag features for previous day data
daily_data['Prev_Open'] = daily_data['Open'].shift(1)
daily_data['Prev_Close'] = daily_data['Close'].shift(1)
daily_data['Prev_High'] = daily_data['High'].shift(1)
daily_data['Prev_Low'] = daily_data['Low'].shift(1)
daily_data['Prev_Volume'] = daily_data['Volume'].shift(1)

# Drop rows with NaN values (as they don't have previous day data)
daily_data.dropna(inplace=True)

# Load the historical trade data (for training purposes)
trade_history_df = pd.read_csv('trade_history.csv')

# Feature Engineering: Extract useful features from the buy/sell data
trade_history_df['Buy Date'] = pd.to_datetime(trade_history_df['Buy Date'])
trade_history_df['Sell Date'] = pd.to_datetime(trade_history_df['Sell Date'])

# Merge daily data with trade history to incorporate lag and current day's features
trade_history_df = pd.merge(trade_history_df, daily_data, left_on='Buy Date', right_on='Date', how='inner')

# Create additional features based on the lag and current day data
trade_history_df['Price Change'] = trade_history_df['Sell Price'] - trade_history_df['Buy Price']
trade_history_df['Holding Period (Months)'] = (trade_history_df['Sell Date'] - trade_history_df['Buy Date']).dt.days / 30
trade_history_df['Price Change %'] = (trade_history_df['Price Change'] / trade_history_df['Buy Price']) * 100

# Target classification: Based on historical trade data, determine Buy (1), Sell (2), or Hold (0)
def classify_action(row):
    if row['Price Change %'] > 10:  # Condition for a profitable trade to Sell
        return 2  # Sell
    elif row['Price Change %'] < -5:  # Condition for a loss to Buy again (lower price)
        return 1  # Buy
    else:
        return 0  # Hold

trade_history_df['Action'] = trade_history_df.apply(classify_action, axis=1)

# Prepare the feature set and target
X = trade_history_df[['Buy Price', 'Open', 'Close', 'Prev_Open', 'Prev_Close', 'Prev_High', 'Prev_Low', 'Prev_Volume']]
y = trade_history_df['Action']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier with Grid Search for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Get the best estimator and evaluate the model performance
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

# Now, let's use the trained model to predict a new action (Buy, Hold, Sell) based on a new day's data
# Prepare the new day's data
today_data = daily_data.iloc[-1]  # Assuming the last row is today's data

new_trade = pd.DataFrame({
    'Buy Price': [today_data['Close']],
    'Open': [today_data['Open']],
    'Close': [today_data['Close']],
    'Prev_Open': [daily_data.iloc[-2]['Open']],
    'Prev_Close': [daily_data.iloc[-2]['Close']],
    'Prev_High': [daily_data.iloc[-2]['High']],
    'Prev_Low': [daily_data.iloc[-2]['Low']],
    'Prev_Volume': [daily_data.iloc[-2]['Volume']]
})

# Standardize the new data
new_trade_scaled = scaler.transform(new_trade)

# Predict action (0: Hold, 1: Buy, 2: Sell)
predicted_action = best_clf.predict(new_trade_scaled)
action_dict = {0: 'Hold', 1: 'Buy', 2: 'Sell'}
print("Predicted Action:", action_dict[predicted_action[0]])

# Apply the function to daily data to get suggested buy prices
def calculate_closest_buy_price(row):
    low = row['Low']
    volume = row['Volume']
    open_price = row['Open']
    close_price = row['Close']
    
    if open_price <= low:
        suggested_buy_price = low
    else:
        suggested_buy_price = low + (volume / 100000)  # Example: Adding a small increment based on volume
    
    return suggested_buy_price

daily_data['Suggested_Buy_Price'] = daily_data.apply(calculate_closest_buy_price, axis=1)

# Display suggested buy prices for today
today_data = daily_data[daily_data['Date'] == pd.to_datetime('today')]
print("Suggested Buy Prices for Today:\n", today_data[['Date', 'Open', 'Close', 'Low', 'High', 'Volume', 'Suggested_Buy_Price']])


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Model Accuracy: 100.00%
Classification Report:
               precision    recall  f1-score   support

           2       1.00      1.00      1.00         7

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7

Predicted Action: Sell
Suggested Buy Prices for Today:
 Empty DataFrame
Columns: [Date, Open, Close, Low, High, Volume, Suggested_Buy_Price]
Index: []
